In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import json
import random

### Data Preparation

In [2]:
def get_dataset(directory):
    feature = []
    label = []
    for foldername in os.listdir(directory):
        folder = os.path.join(directory, foldername)
        if os.path.isdir(folder):
            files = os.listdir(folder)
            for filename in files:
                rel_path = os.path.join(directory, foldername, filename)
                temp_label = filename.split('.')[0].split('_')[0]
                if 'a' in temp_label:
                    label = np.append(label,'alcoholic')
                else:
                    label = np.append(label,'control')
                
                df_data = np.loadtxt(rel_path, delimiter=",")
                feature.append(df_data)
    return feature, label

### Model Definition

In [6]:
def create_model():
    input_layer = keras.Input(shape=(512, 1))

    x = layers.Conv1D(filters=8, kernel_size=8, strides=2, activation="relu")(input_layer)
    x = layers.MaxPooling1D(pool_size=2)(x)

    x = layers.Conv1D(filters=4, kernel_size=4, strides=2, activation="relu")(x)
    x = layers.MaxPooling1D(pool_size=2)(x)

    x = layers.Flatten()(x)

    x = layers.Dense(4096, activation="relu")(x)

    x = layers.Dense(2048, activation="relu")(x)

    output_layer = layers.Dense(2, activation="softmax")(x)

    return keras.Model(inputs=input_layer, outputs=output_layer)

In [4]:
# loading extracted feature & label
x_test, y_test = get_dataset('../SMNI_CMI_TEST_FEATURE')
x_train, y_train = get_dataset('../SMNI_CMI_TRAIN_FEATURE')

co2a0000364
co2a0000365


array(['alcoholic', 'alcoholic', 'alcoholic', 'alcoholic', 'alcoholic',
       'alcoholic', 'control', 'alcoholic', 'alcoholic', 'alcoholic',
       'alcoholic', 'alcoholic', 'alcoholic', 'alcoholic'], dtype='<U32')